<a href="https://colab.research.google.com/github/jg4726/public-file/blob/main/Transformer_stock_market_stock_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.random.set_seed(42)
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

device = 'GPU' if 'GPU' in tf.test.gpu_device_name() else 'CPU/TPU'
print('Device:', device)

import os, gc, random, datetime
if device == 'GPU':
    import cudf
    import cupy as cp
import pandas as pd
import numpy as np
# import janestreet
import xgboost as xgb
import datatable as dtable
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from joblib import dump, load
from time import time
from numba import njit

print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Device: CPU/TPU


Tensorflow version 2.7.0


In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
MIXED_PRECISION = False
XLA_ACCELERATE = True

if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

Accelerated Linear Algebra enabled


In [ ]:
%%time

print('Loading...')
train = dtable.fread('jane_street_train.jay').to_pandas()
features = [c for c in train.columns if 'feature' in c]

print('Filling...')
train = train.query('weight > 0').reset_index(drop = True)
train[features] = train[features].fillna(method = 'ffill').fillna(0)
train['action'] = (train['resp'] > 0).astype('int')

print('Finish.')

Loading...
Filling...
Finish.
Wall time: 7.1 s


In [ ]:
if 'date' in features:
    print('yes')

In [ ]:
train.columns.values

array(['date', 'weight', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp',
       'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13',
       'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_23', 'feature_24', 'feature_25',
       'feature_26', 'feature_27', 'feature_28', 'feature_29',
       'feature_30', 'feature_31', 'feature_32', 'feature_33',
       'feature_34', 'feature_35', 'feature_36', 'feature_37',
       'feature_38', 'feature_39', 'feature_40', 'feature_41',
       'feature_42', 'feature_43', 'feature_44', 'feature_45',
       'feature_46', 'feature_47', 'feature_48', 'feature_49',
       'feature_50', 'feature_51', 'feature_52', 'feature_53',
       'feature_54', 'feature_55', 'feature_56', 'feature_57',
       'feature_58', 'featu

*Base Transformer structure from https://www.tensorflow.org/tutorials/text/transformer, modified with Swish activation function.*

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b = True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis = -1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads):
        
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm = [0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm = [0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
        return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
    
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation = 'swish'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads, dff, rate = 0.1):
        
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training = training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

class TransformerEncoder(tf.keras.layers.Layer):
    
    def __init__(self, num_layers, d_model, num_heads, dff, 
                 maximum_position_encoding, rate = 0.1):
        
        super(TransformerEncoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dff = dff
        self.maximum_position_encoding = maximum_position_encoding
        self.rate = rate

#         self.pos_encoding = positional_encoding(self.maximum_position_encoding, 
#                                                 self.d_model)
#         self.embedding = tf.keras.layers.Dense(self.d_model)
        self.pos_emb = tf.keras.layers.Embedding(input_dim = self.maximum_position_encoding, 
                                                 output_dim = self.d_model)

        self.enc_layers = [EncoderLayer(self.d_model, self.num_heads, self.dff, self.rate) 
                           for _ in range(self.num_layers)]

        self.dropout = tf.keras.layers.Dropout(self.rate)
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'num_layers': self.num_layers,
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'maximum_position_encoding': self.maximum_position_encoding,
            'dropout': self.dropout,
        })
        return config

    def call(self, x, training, mask = None):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
#         x += self.pos_encoding[:, :seq_len, :]
#         x = self.embedding(x)
        positions = tf.range(start = 0, limit = seq_len, delta = 1)
        x += self.pos_emb(positions)

        x = self.dropout(x, training = training)

        for i in range(self.num_layers):

            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [ ]:
def create_transformer_model(num_columns, num_labels, num_layers, d_model, num_heads, dff, window_size, dropout_rate, weight_decay, label_smoothing, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (window_size, num_columns))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dense(d_model)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('swish')(x)
    x = tf.keras.layers.SpatialDropout1D(dropout_rate)(x)
    x = TransformerEncoder(num_layers, d_model, num_heads, dff, window_size, dropout_rate)(x)
    out = tf.keras.layers.Dense(num_labels, activation = 'sigmoid')(x[:, -1, :])
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tfa.optimizers.AdamW(weight_decay = weight_decay, learning_rate = learning_rate),
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing), 
                  metrics = tf.keras.metrics.AUC(name = 'AUC'), 
                 )
    
    return model

In [ ]:
batch_size = 4096 * strategy.num_replicas_in_sync
num_layers = 1
d_model = 96
num_heads = 1
dff = 64
window_size = 3
dropout_rate = 0.15
weight_decay = 0
label_smoothing = 1e-2
learning_rate = 1e-3 * strategy.num_replicas_in_sync
verbose = 1

In [ ]:
with strategy.scope():
    model = create_transformer_model(len(features), 1, num_layers, d_model, num_heads, dff, window_size, dropout_rate, weight_decay, label_smoothing, learning_rate)
model.summary()

K.clear_session()
del model
rubbish = gc.collect()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 130)]          0         
                                                                 
 batch_normalization (BatchN  (None, 3, 130)           520       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 3, 96)             12576     
                                                                 
 batch_normalization_1 (Batc  (None, 3, 96)            384       
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 3, 96)             0         
                                                                 
 spatial_dropout1d (SpatialD  (None, 3, 96)            0     

In [ ]:
# Use Tensorflow Dataset
def prepare_dataset(X, y, window_size, batch_size, mode = 'training'):
    x_ds = tf.data.Dataset.from_tensor_slices(X) 
    y_ds = tf.data.Dataset.from_tensor_slices(y[window_size - 1:])
    x_ds = x_ds.window(window_size, shift = 1, drop_remainder = True)
    x_ds = x_ds.flat_map(lambda window: window.batch(window_size))
    dataset = tf.data.Dataset.zip((x_ds, y_ds))
    if mode == 'training':
        buffer_size = batch_size * 8
        dataset = dataset.repeat()
        dataset = dataset.shuffle(buffer_size, reshuffle_each_iteration = True)
        dataset = dataset.batch(batch_size)#, drop_remainder = True
    elif mode == 'validation':
        dataset = dataset.batch(batch_size)
        dataset = dataset.cache() 
    elif mode == 'testing':
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Use Numpy [may cause Out-of-Memory (OOM) error]
def rolling_window(a, shape):  # rolling window for 2D array
    s = (a.shape[0] - shape[0] + 1,) + (a.shape[1] - shape[1] + 1,) + shape
    strides = a.strides + a.strides
    return np.squeeze(np.lib.stride_tricks.as_strided(a, shape = s, strides = strides), axis = 1)

In [ ]:
X_tr = train.loc[train['date'] <= 450, features].values
y_tr = train.loc[train['date'] <= 450, 'action'].values

# X_tr2 = train.loc[(train['date'] >= 303) & (train['date'] <= 367), features].values
# y_tr2 = train.loc[(train['date'] >= 303) & (train['date'] <= 367), 'action'].values

X_val = train.loc[train['date'] > 450, features].values
y_val = train.loc[train['date'] > 450, 'action'].values

rubbish = gc.collect()

In [ ]:
print(len(X_tr))
# print(len(X_tr2))
print(len(X_val))


1748599
232688


In [ ]:
X_tr = rolling_window(X_tr, (window_size, len(features)))
X_val = rolling_window(X_val, (window_size, len(features)))
y_tr = y_tr[window_size - 1:]
y_val = y_val[window_size - 1:]
# X_tr2 = rolling_window(X_tr2, (window_size, len(features)))
# y_tr2 = y_tr2[window_size - 1:]

261526


In [ ]:
start_time_fold = time()

ckp_path = 'JSTransformer.hdf5'
with strategy.scope():
    model = create_transformer_model(len(features), 1, num_layers, d_model, num_heads, dff, window_size, dropout_rate, weight_decay, label_smoothing, learning_rate)
rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 3, verbose = verbose, 
                        min_delta = 1e-4, mode = 'max')
ckp = ModelCheckpoint(ckp_path, monitor = 'val_AUC', verbose = 0, 
                      save_best_only = True, save_weights_only = True, mode = 'max')
es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 7, mode = 'max', 
                   baseline = None, restore_best_weights = True, verbose = 0)
history = model.fit(X_tr, y_tr, validation_data = (X_val, y_val), batch_size = batch_size,
                    epochs = 100, callbacks = [rlr, ckp, es], verbose = verbose)
hist = pd.DataFrame(history.history)
print(f'[{str(datetime.timedelta(seconds = time() - start_time_fold))[0:7]}] ROC AUC:\t', hist['val_AUC'].max())

K.clear_session()
del model, X_tr, y_tr
rubbish = gc.collect()

Epoch 1/100
427/427 [==============================] - 66s 151ms/step - loss: 0.6996 - AUC: 0.5120 - val_loss: 0.6917 - val_AUC: 0.5301 - lr: 0.0010
Epoch 2/100
427/427 [==============================] - 64s 150ms/step - loss: 0.6921 - AUC: 0.5248 - val_loss: 0.6914 - val_AUC: 0.5309 - lr: 0.0010
Epoch 3/100
427/427 [==============================] - 64s 151ms/step - loss: 0.6916 - AUC: 0.5280 - val_loss: 0.6914 - val_AUC: 0.5326 - lr: 0.0010
Epoch 4/100
427/427 [==============================] - 65s 152ms/step - loss: 0.6912 - AUC: 0.5308 - val_loss: 0.6913 - val_AUC: 0.5349 - lr: 0.0010
Epoch 5/100
427/427 [==============================] - 63s 149ms/step - loss: 0.6908 - AUC: 0.5327 - val_loss: 0.6915 - val_AUC: 0.5358 - lr: 0.0010
Epoch 6/100
427/427 [==============================] - 65s 151ms/step - loss: 0.6905 - AUC: 0.5339 - val_loss: 0.6912 - val_AUC: 0.5375 - lr: 0.0010
Epoch 7/100
427/427 [==============================] - 65s 153ms/step - loss: 0.6902 - AUC: 0.5351 - val_l